To close the NetLogo environment, uncomment the following line and run.

In [43]:
# netlogo.kill_workspace()

Import the following packages:

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys # used to run other scripts
import numpy as np
from random import randint

Install the following packages: 

For installation instructions, follow https://pynetlogo.readthedocs.io/en/latest/install.html

In [1]:
import pyNetLogo
import jpype

Fill in the path to the right directory and start up NetLogo:

In [3]:
# For windows:
os.chdir('D:\Your\\Directory\\Path')
netlogo = pyNetLogo.NetLogoLink(gui=True)

#For linux/hpc:
# netlogo = pyNetLogo.NetLogoLink(gui=False,netlogo_home="NetLogo611",netlogo_version='6.1',jvm_home='/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.222.b10-1.el7_7.x86_64/jre/lib/amd64/server/libjvm.so')


In [2]:
# Load the NetLogo model:
netlogo.load_model('Camp expansion model - PyNetlogo model.nlogo')

# # When performing the sensitivity analysis, use the following instead:
# netlogo.load_model('Camp expansion model SA -  PyNetlogo model')

### Optimization function

Below, the optimization scripts are defined. They can be all run at once. Four .csv files will be created, one for each model.

#### Define the experiments:

In [5]:
#To make sure the right folder is used:
os.getcwd()

'C:\\Users\\meyke\\Downloads'

Read the excel file that defines the global parameters. 

When performing a sensitivity analysis, make sure that the right NetLogo model is used, and the excel specifies all parameters that can be varied in the sensitivity analysis.

In [6]:
#Read the excel file that holds the experiment setup:
myexperiments = pd.read_excel('experiments_globals_longlist.xlsx')

#Every experiment in the excel gets the same random-seed:
myexperiments['random-seed']= 42 #Fill in the desired random-seed.

When a different random-seed for every experiment is desired, uncomment the following:

In [ ]:
# for i in range(len(myexperiments)):
#     myexperiments.loc[i, 'random-seed'] = randint(0, 2147483647) # this range is chosen, based on the possible random-seeds in NetLogo.

### 1.1 P-median optimization

#### Scenario 1: not taking into account expected future growth

In [15]:
def pmedian_optimization():

    x = netlogo.report('map [s -> [xcor] of s] sort shelters')
    y = netlogo.report('map [s -> [ycor] of s] sort shelters')
    c = netlogo.report('map [s -> [available-here] of s] sort shelters')
    d = netlogo.report('map [s -> [covered?] of s] sort shelters')
    
    if (netlogo.report('count facilities')) > 0.0:
        hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
        hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    

    coord_shelters2 = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # all shelters
    coord_shelters = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # now all shelters, but will drop the covered shelters and shelters at full locations
    coord_uncovered = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # all uncovered shelters
    coord_facilities = pd.DataFrame()

    if (netlogo.report('count facilities')) > 0.0:
        coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy}) # all facilities
            
    no_availability = coord_shelters[ (coord_shelters['patch_availability'] <= 0)].index 
    coord_shelters.drop(no_availability, inplace = True)
    already_covered = coord_shelters [(coord_shelters['covered?'] == True)].index

    coord_shelters = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d})
    coord_shelters = coord_shelters.append(coord_facilities, ignore_index = True)

    coord_shelters.drop(no_availability, inplace = True) # Here, the shelters are dropped that have no available space. These are no longer possible facility locations
    coord_shelters.drop(already_covered, inplace = True) # Here, the shelters are dropped that are already covered. Placing a facility here will not minimize the average distance. 


    FacilityDF = pd.DataFrame()

    for i in coord_shelters.index: #for every demand point, calculate distance to possible facility locations using the logic of Pythagoras (but without the square-root)
        b = ((coord_shelters2['xcor'] - coord_shelters.loc[i, 'xcor']) * (coord_shelters2['xcor'] - coord_shelters.loc[i, 'xcor']) +
             (coord_shelters2['ycor'] - coord_shelters.loc[i, 'ycor']) * (coord_shelters2['ycor'] - coord_shelters.loc[i, 'ycor']))
        FacilityDF[str(i)] = b # Store these distances in FacilityDF
    testFacilityDF = FacilityDF.copy()

    
    for i in range(len(coord_facilities)): #Only maintain the options that realize a distance below 40 patches (note: square-root in Pythagoras is skipped in previous step)
        sortedcolumn = pd.DataFrame()
        sortedcolumn = testFacilityDF.iloc[:, (- i + 1)].sort_values(ascending = True)
        sortedcolumn = sortedcolumn.where(sortedcolumn < 1600).dropna()
        
        if len(sortedcolumn) > 22:
            sortedcolumn = sortedcolumn.head(22)
        testFacilityDF = testFacilityDF.drop(sortedcolumn.index)


    if len(coord_facilities) != 0: 
        testFacilityDF = testFacilityDF.drop(testFacilityDF.iloc[:,-(len(coord_facilities)):], axis = 1)

    #Create an empty dataframe:
    DistanceDF2 = pd.DataFrame()
    DistanceDF2['avg_dist'] = []

    for i in testFacilityDF.columns: #For every possible facility point
        for j in testFacilityDF.columns: #For second possible facility point
            dist = 0 
            location = str(i) + '-' + str(j) #Name according to the two facility locations
            
            for k in testFacilityDF.index: #For every demand point, determine distance and ...
                dist = dist + min(testFacilityDF.loc[k,str(i)], testFacilityDF.loc[k,str(j)]) # ... find the closest facility
            
            avg_dist = dist/len(testFacilityDF) # Find the average demand-weighted distance, if everyone goes to the nearest facility
            
            DistanceDF2.loc[location,'avg_dist'] = avg_dist #Store this in the dataframe.
        
    DistanceDF2 = DistanceDF2.sort_values('avg_dist')
    

    aa = int(netlogo.report('count facilities'))
    ab = int(netlogo.report('count shelters')) #Here: count all shelters, also the predicted ones, because you want to place facilities also for the predicted shelters.
    initialcapacity = int(netlogo.report('initial-capacity'))
    ac = aa / ab

    if ac <= (1 / initialcapacity): # Only when the threshold to realize a new healthcare facility is exceeded, new facilities are created.
        netlogo.command('create-random-facilities')

        # This block obtains the indexes of the coordinates where the health facilities should be located,
        # and stores these in a new dataframe: Healthfacs.
        z = DistanceDF2.index[1].split('-')
        m = netlogo.report('map [s -> [who] of s] sort facilities')
        Healthfacs = pd.DataFrame()

        for i in range(0,len(z)):
            a = int(z[i])
            Healthfacs = Healthfacs.append(coord_shelters2.loc[a])

        Healthfacs.reset_index(inplace = True)
        for j in range(0,len(z)):
            Healthfacs.loc[j, 'who'] = m[-(j+1)]  # Find the facilities that are just created (newest two) and determine where they should be located

    

        #Write these attributes to Netlogo, setting the xcor and ycor of the latest facilities.
        netlogo.write_NetLogo_attriblist(Healthfacs[['who','xcor','ycor']], 'facility')
        netlogo.command('ask facilities [ask patch-here [set available 0] ask shelters-here [set available-here 0]]')


Create empty dataframes that will store the simulation results:

In [7]:
reporters = []
report = []
results = []

In [8]:
def pmedian():
# # P-median optimization:

    pmedian_optimization() #Performs the optimization, in the block of code above.

    # After an optimization, the agents are mapped again and new links are created, because new facilities may have been created.
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})
    netlogo.command('link-facility-shelters')
    
    # Run four ticks and repeat the results
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters','sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks',
                                     'Pr1:Road-proximity','Pr2:Neighbour-proximity','Pr3:Healthcare-proximity','space-variable','prediction-accuracy','future-regression'
                                   ],4)
    return report

In [10]:
def run_simulation(experiment):
    netlogo.command('setup-world') # Sets up the camp layout
    '''run a netlogo model

    Parameters
    ----------
    experiments : dict

    '''
    reporters = []

    #Set the input parameters that are defined in the experiment file:
    for key, value in experiment.items():
        if key == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))
    
    netlogo.command('setup') # Setup the first shelters in the camp:
            
    while (netlogo.report('ticks')) < 94: # The simulation captures 94 weeks (august 2017 till June 2019)
        reporters.append(pmedian())

#         If it is desired to print the results during the simulation, uncomment the following lines:
#         print('The function returns:') 
#         print(reporters)
                
    return reporters

myresults1 = pd.DataFrame()

for i, row in myexperiments.iterrows():
    results.append(run_simulation(row))
    
    # Store the results in a dataframe: resultsDF:
    resultsDF1 = pd.DataFrame(results)


for i in resultsDF1.index:
    for j in resultsDF1.columns:
        myresults1 = myresults1.append(resultsDF1.iloc[i,j])

# Write the results to a .csv file, called: Pmedian_experiment1.csv
myresults1.to_csv('Pmedian_experiment1.csv') 

Setup complete
The function returns:
[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ]
The function returns:


C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ,      average-distance covered-shelters sumshelters predicted-shelters  \
10.0          42.9538               44         115                  0   
11.0          35.4567               89         119                  0   
12.0          32.7285               94

C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ,      average-distance covered-shelters sumshelters predicted-shelters  \
10.0          42.9538               44         115                  0   
11.0          35.4567               89         119                  0   
12.0          32.7285               94

...

[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ,      average-distance covered-shelters sumshelters predicted-shelters  \
10.0          42.9538               44         115                  0   
11.0          35.4567               89         119                  0   
12.0          32.7285               94

C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ,      average-distance covered-shelters sumshelters predicted-shelters  \
10.0          42.9538               44         115                  0   
11.0          35.4567               89         119                  0   
12.0          32.7285               94

C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                  0   
7.0          42.3168               44         100                  0   
8.0           42.833               46         105                  0   
9.0           42.743               46         113                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0        false     6  
7.0                  27              56              0        false     7  
8.0                  17              41              0        false     8  
9.0                  21              53              0        false     9  ,      average-distance covered-shelters sumshelters predicted-shelters  \
10.0          42.9538               44         115                  0   
11.0          35.4567               89         119                  0   
12.0          32.7285               94

### 1.2 P-median future optimization

#### Scenario 2: while taking into account expected future growth

In [16]:
reporters = []
report = []
results = []

In [17]:
#The P-median optimization, with Future.

def pmedian_future():
    netlogo.command('set future-modus True') # This determines that the prediction starts.
    netlogo.command('create-new-shelters') # This creates predicted shelters.
    netlogo.command('ask shelters with [prediction = True][set size 20]') # This is purely for visibility of what happens
    
    pmedian_optimization() # Performs the facility location optimization
    
    # to calculate the performance of the predicted situation and the real situation:
    # link-facility-shelters now, calculate the average-distance and coverage and then re-link afterwards.
    netlogo.command('link-facility-shelters')
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks'],1)
    
    # When predicted shelters are created, their impact on the availability of the patches around them should be canceled:
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius (0.5 * radius) [set available (available + 1) ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius 3 [set available 2 ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patch-here [set available 2 ]]')

    #if prediction-accuracy == 100, the predicted shelters should stay, but their links should be removed
    if netlogo.report('prediction-accuracy') == 100:
        netlogo.command('ask shelters with [prediction = True] [ask my-out-links [die]]')
    
    # if prediction accuracy == 0, the predicted shelters should be removed.
    else: 
        netlogo.command('ask shelters with [prediction = True] [die]')
    netlogo.command('set future-modus False') # Stop the future-modus
    netlogo.command('set last-count (count shelters with [prediction = False])') #Count the real shelters
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})

    netlogo.command('link-facility-realshelters') #Link only real shelters to facilities, not the predicted shelters.
    reportrepeat = [netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks'],4)]
    report = report.append(reportrepeat)
    return report

In [18]:
def run_simulation(experiment):
    netlogo.command('setup-world')
    '''run a netlogo model

    Parameters
    ----------
    experiments : dict

    '''
    reporters = []

    #Set the input parameters
    for key, value in experiment.items():
        if key == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))
    
    netlogo.command('setup')    

            
    while (netlogo.report('ticks')) < 94:
        print('The function returns:')
        reporters.append(pmedian_future())
        
#         If it is desired to print the results during the simulation, uncomment the following lines:
#         print('The function returns:') 
#         print(reporters)             
                
    return reporters

myresults2 = pd.DataFrame()

for i, row in myexperiments.iterrows():
    results.append(run_simulation(row))
    
    #store this in a dataframe ; myresults:
    resultsDF2 = pd.DataFrame(results)

    
for i in resultsDF2.index:
    for j in resultsDF2.columns:
        myresults2 = myresults2.append(resultsDF2.iloc[i,j])

# Write the results to a .csv file, called: Pmedian_future_experiment2.csv
myresults2.to_csv('Pmedian_future_experiment2.csv')

The function returns:
The function returns:


C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


The function returns:


C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


The function returns:


...

The function returns:


C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


The function returns:


C:\Users\meyke\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


one experiment is finished


### 2.1 Maximal covering optimization

#### Scenario 1: not taking into account expected future growth

In [6]:
#From Python script:

def maxcovering_optimization():

    x = netlogo.report('map [s -> [xcor] of s] sort shelters')
    y = netlogo.report('map [s -> [ycor] of s] sort shelters')
    c = netlogo.report('map [s -> [available-here] of s] sort shelters')
    d = netlogo.report('map [s -> [covered?] of s] sort shelters')

    if (netlogo.report('count facilities')) > 0.0:
        hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
        hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')


    coord_shelters2 = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # all shelters
    coord_shelters = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # all shelters, will drop the ones where the patch is already full.
    coord_uncovered = pd.DataFrame({'xcor':x, 'ycor':y, 'patch_availability':c, 'covered?':d}) # all uncovered shelters

    if (netlogo.report('count facilities')) > 0.0:
        coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})
    else:
        coord_facilities = pd.DataFrame() # all existing facilities
        
    no_availability = coord_shelters[ (coord_shelters['patch_availability'] <= 0)].index
    coord_shelters.drop(no_availability, inplace=True) # Here, drop the shelters at locations where the patch is full.

    covered = coord_uncovered [(coord_uncovered['covered?'] == True)].index 
    coord_uncovered.drop(covered, inplace = True) # Here, drop the shelters that are already covered by a healthcare facility.

    # # In this algorithm, do NOT append existing facilities, because the create-facilities command comes after the link-facility-shelter command,
    # # which means that already the closest shelters are linked to the existing facilities, so optimization must only happen for the
    # # uncovered shelters, for which a new facility is needed.

    FacilityDF = pd.DataFrame()

    for i in coord_uncovered.index: #for every demand point, calculate distance to possible facility locations (following Pythagoras, but skipping the square-root)
            b = ((coord_shelters['xcor'] - coord_uncovered.loc[i, 'xcor']) * (coord_shelters['xcor'] - coord_uncovered.loc[i, 'xcor']) +
             (coord_shelters['ycor'] - coord_uncovered.loc[i, 'ycor']) * (coord_shelters['ycor'] - coord_uncovered.loc[i, 'ycor']))
            FacilityDF[str(i)] = b # and store is in a DataFrame: FacilityDF.

    FacilityDF = FacilityDF.where(FacilityDF < 1600) # Drop all options that are more than 40 patches away (stil skipped Pythagoras' square-root)
    BestLocation = pd.DataFrame() # Best location will store the number of uncovered shelters that can be covered by a new facility in a specific location.
    BestLocation['count_<1600'] = FacilityDF.count(axis = 0).sort_values(ascending = False) # if you take axis=1 here, it also searches the closest location for the two present facilities. That is wrong. Therefore, it must be axis = 0.

    FacilityDF2 = BestLocation.where(BestLocation == (max(BestLocation['count_<1600']))) # Find the facility location that maximizes the coverage of uncovered shelters
    FacilityDF2 = FacilityDF2.dropna() # Drop all others possible locations.
    FacilityDF2

    x = int(netlogo.report('facilities_to_create'))


    Healthfacs = pd.DataFrame()

#     If more than 2 facility locations are optimal, the location is chosen that is furthest away from all other facilities:
    if len(FacilityDF2) > x: 
        BestLocation2 = pd.DataFrame()
        for i in range(len(FacilityDF2)):
            a = int(FacilityDF2.index[i])
            BestLocation2 = BestLocation2.append(coord_shelters2.iloc[a]) 

        BestLocation2.reset_index(inplace = True)
        facilityname = []

        for i in coord_facilities.index: # Here, the distance to all other facilities is determined (using Pythagoras' theorem)
            b = ((BestLocation2['xcor'] - coord_facilities.loc[i, 'xcor']) * (BestLocation2['xcor'] - coord_facilities.loc[i, 'xcor'])) + ((BestLocation2['ycor'] - coord_facilities.loc[i, 'ycor']))
            BestLocation2['facility'+str(i)] = b**(1/2)
            facilityname.append('facility'+str(i))
        BestLocation2['avg_dist_to_facility'] =BestLocation2[facilityname].mean(axis=1)
        BestLocation2.sort_values('avg_dist_to_facility', ascending = False, inplace = True)
        BestLocation2.reset_index(drop = True, inplace = True)

    # In this case, the indexes are read from the column BestLocation2['index']:
        for i in range(x):
            a = int(BestLocation2['index'][i])
            Healthfacs = Healthfacs.append(coord_shelters2.iloc[a]) 

    # If there is no more than 2 healthfacilities that maximize coverage, the following is used:
    else:
        for i in range(x):
            a = int(BestLocation.index[i])
            Healthfacs = Healthfacs.append(coord_shelters2.iloc[a]) 

#     Determine whether the threshold to create new facilities is exceeded:
    aa = int(netlogo.report('count facilities'))
    ab = int(netlogo.report('count shelters'))
    initialcapacity = int(netlogo.report('initial-capacity'))
    ac = aa / ab

    if ac <= (1 / initialcapacity): #If the threshold is exceeded, new facilities are created:
        netlogo.command('create-random-facilities')
        
        #Adding a column with a 'who' index, corresponding to the latest created facilities.
        Healthfacs.reset_index(inplace = True)

        m = netlogo.report('map [s -> [who] of s] sort facilities')

        for j in range(x):
            Healthfacs.loc[j, 'who'] = m[-(j+1)]

        #Write these attributes to Netlogo, setting the xcor and ycor of the latest facilities.
        netlogo.write_NetLogo_attriblist(Healthfacs[['who','xcor','ycor']], 'facility')
        netlogo.command('ask facilities [ask patch-here [set available 0] ask shelters-here [set available-here 0]]')


In [8]:
reporters = []
report = []
results = []

When using the maximal covering optimization, two facilities are not placed simultaneously anymore. Only the first time, two facilities are placed simultaneously, then (after 7 ticks in the model) the optimization is performed twice. This is done, because otherwise the two new facilities are often located very near to each other. 

In [46]:
#while ticks < 7: 
def maxcovering7():
    maxcovering_optimization()
    
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})
    
    netlogo.command('link-facility-shelters')
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks'],4)
    return report

#while ticks > 7:
def maxcovering94():
    netlogo.command('set facilities_to_create 1')
    maxcovering_optimization()
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})

    maxcovering_optimization()
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})
    
    netlogo.command('link-facility-shelters')
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks'],4)
    return report

In [47]:
def run_simulation(experiment):
    netlogo.command('setup-world')
    '''run a netlogo model

    Parameters
    ----------
    experiments : dict

    '''
    reporters = []

    #Set the input parameters
    for key, value in experiment.items():
        if key == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))
    
    netlogo.command('setup')    

            
    while (netlogo.report('ticks')) < 7:
        reporters.append(maxcovering7())
    
    while (netlogo.report('ticks')) < 94:
        reporters.append(maxcovering94())
        
    return reporters


myresults3 = pd.DataFrame()

for i, row in myexperiments.iterrows():
    results.append(run_simulation(row))
    #store this in a dataframe: resultsDF3
    resultsDF3 = pd.DataFrame(results)
    
for i in resultsDF3.index:
    for j in resultsDF3.columns:
        myresults3 = myresults3.append(resultsDF3.iloc[i,j])
        
# Write the results to a .csv file, called: Maxcovering_experiment3.csv
myresults3.to_csv('Maxcovering_experiment3.csv')

one experiment is done


### 2.2 Maximal covering optimization

#### Scenario 2: while taking into account expected future growth

In [37]:
reporters = []
report = []
results = []

In [40]:
def maxcovering7_future():
   # # P-median optimization:
    netlogo.command('set future-modus True')  # This determines that the prediction starts.
    netlogo.command('create-new-shelters') # This creates predicted shelters.
    netlogo.command('ask shelters with [prediction = True][set size 20]') # This is purely for visibility of what is happening

    maxcovering_optimization()
    # to calculate the performance of the predicted situation and the real situation:
    # link-facility-shelters now, calculate the average-distance and coverage and then re-link afterwards.
    netlogo.command('link-facility-shelters')
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks',
                                     'Pr1:Road-proximity','Pr2:Neighbour-proximity','Pr3:Healthcare-proximity','space-variable','prediction-accuracy','future-regression'],1)
    
    # When predicted shelters are created, their impact on the availability of the patches around them should be canceled:
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius (0.5 * radius) [set available (available + 1) ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius 3 [set available 2 ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patch-here [set available 2 ]]')

    #if prediction-accuracy == 100, the predicted shelters should stay, but their links should be removed
    if netlogo.report('prediction-accuracy') == 100:
        netlogo.command('ask shelters with [prediction = True] [ask my-out-links [die]]')
    
    # if prediction accuracy == 0, the predicted shelters should be removed.
    else:
        netlogo.command('ask shelters with [prediction = True] [die]')
    netlogo.command('set future-modus False') # Stop the future-modus
    netlogo.command('set last-count (count shelters with [prediction = False])')
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})  
    
    netlogo.command('link-facility-realshelters') #Link only real shelters to facilities, not the predicted shelters.
    reportrepeat = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks',
                                     'Pr1:Road-proximity','Pr2:Neighbour-proximity','Pr3:Healthcare-proximity','space-variable','prediction-accuracy','future-regression'],4)
    report = report.append(reportrepeat)
    return report

        
#while ticks > 7:
def maxcovering94_future():
    
    netlogo.command('set facilities_to_create 1')
    
    netlogo.command('set future-modus True')
    netlogo.command('create-new-shelters')
    netlogo.command('ask shelters with [prediction = True][set size 20]')

    maxcovering_optimization()
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})

    maxcovering_optimization()
    hfx = netlogo.report('map [s -> [xcor] of s] sort facilities')
    hfy = netlogo.report('map [s -> [ycor] of s] sort facilities')
    coord_facilities = pd.DataFrame({'xcor':hfx, 'ycor':hfy})
    
    # to calculate the performance of the predicted situation and the real situation:
    # link-facility-shelters now, calculate the average-distance and coverage and then re-link afterwards.
    netlogo.command('link-facility-shelters')
    report = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks',
                                     'Pr1:Road-proximity','Pr2:Neighbour-proximity','Pr3:Healthcare-proximity','space-variable','prediction-accuracy','future-regression'],1)
    # That was this part.
    
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius (0.5 * radius) [set available (available + 1) ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patches in-radius 3 [set available 2 ]]')
    netlogo.command('ask shelters with [prediction = True] [ask patch-here [set available 2 ]]')

    #if prediction-accuracy == 100, don't die, but only links die:
    if netlogo.report('prediction-accuracy') == 100:
        netlogo.command('ask shelters with [prediction = True] [ask my-out-links [die]]')
    else:
        netlogo.command('ask shelters with [prediction = True] [die]')
    netlogo.command('set future-modus False')
    netlogo.command('set last-count (count shelters with [prediction = False])')
    
    netlogo.command('link-facility-realshelters')
    reportrepeat = netlogo.repeat_report(['average-distance','covered-shelters','sumshelters','predicted-shelters',
                                     'sumwaiting-patients','sumovercapacity','unusedcapacity','future-modus','ticks',
                                     'Pr1:Road-proximity','Pr2:Neighbour-proximity','Pr3:Healthcare-proximity','space-variable','prediction-accuracy','future-regression'],4) 
    
    report = report.append(reportrepeat)
    return report

In [42]:
# original Notebook version:
# works
def run_simulation(experiment):
    netlogo.command('setup-world')
    '''run a netlogo model

    Parameters
    ----------
    experiments : dict

    '''
    reporters = []

    #Set the input parameters
    for key, value in experiment.items():
        if key == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))
    
    netlogo.command('setup')    

            
    while (netlogo.report('ticks')) < 7:
        reporters.append(maxcovering7_future())

    while (netlogo.report('ticks')) < 94:
        reporters.append(maxcovering94_future())

    return reporters

myresults4 = pd.DataFrame()

for i, row in myexperiments.iterrows(): 
    results.append(run_simulation(row))
    
    #store this in a dataframe: resultsDF4:
    resultsDF4 = pd.DataFrame(results)
    
    
for i in resultsDF4.index:
    for j in resultsDF4.columns:
        myresults4 = myresults4.append(resultsDF4.iloc[i,j])

myresults4.to_csv('Maxcovering_future_experiment4.csv')

reporters is <7:
[    average-distance covered-shelters sumshelters predicted-shelters  \
6.0                0                0           0                 20   
6.0          53.4678               33          74                 20   
7.0          54.4583               45          86                  0   
8.0          55.6156               46          96                  0   
9.0          55.8134               44         109                  0   

    sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
6.0                   0               0              0         true     6  
6.0                   0              50              0        false     6  
7.0                  25              41             -1        false     7  
8.0                  11              39              0        false     8  
9.0                  22              52              0        false     9  ]
this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-she

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
18.0           31.679              136         162                 20   
19.0          34.9888              157         165                  0   
20.0          36.0253              157         165                  0   
21.0          34.3886              156         168                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
18.0                   6              25             23        false    18  
19.0                  18               8             19        false    19  
20.0                  11               8             19        false    20  
21.0                  13              10             22        false    21  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
18.0          37.1337      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
26.0           29.924              159         171                 20   
27.0          33.6957              163         172                  0   
28.0          33.6755              166         175                  0   
29.0          33.0111              167         184                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
26.0                   4              10             25        false    26  
27.0                   0               8             36        false    27  
28.0                   9               8             34        false    28  
29.0                  12              14             33        false    29  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
26.0          34.6316      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
34.0            27.98              168         201                 20   
35.0          32.0586              184         201                  0   
36.0          34.1556              188         201                  0   
37.0          32.1723              192         204                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
34.0                  16              29             58        false    34  
35.0                  15              13             60        false    35  
36.0                  10              11             56        false    36  
37.0                  11               9             51        false    37  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
34.0           30.718      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
38.0          29.9416              178         208                 20   
39.0          33.7009              195         210                  0   
40.0          35.2682              196         210                  0   
41.0          33.0513              190         212                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
38.0                  10              33             54        false    38  
39.0                  17              14             50        false    39  
40.0                   7              13             47        false    40  
41.0                   8              18             54        false    41  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
38.0          32.2865      

...

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
74.0          29.3704              192         236                 20   
75.0          32.9838              203         236                  0   
76.0          35.0918              209         236                  0   
77.0          36.8653              217         236                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
74.0                  13              47             64        false    74  
75.0                   7              27             62        false    75  
76.0                  27              21             55        false    76  
77.0                  13              17             47        false    77  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
74.0          32.3197      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
78.0          31.7949              201         236                 20   
79.0           33.608              214         236                  0   
80.0          35.7715              220         236                  0   
81.0          37.0387              224         236                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
78.0                  15              42             63        false    78  
79.0                  26              19             57        false    79  
80.0                  13              14             49        false    80  
81.0                  21              11             44        false    81  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
78.0          39.2781      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
82.0          31.5536              197         236                 20   
83.0           33.779              213         236                  0   
84.0          34.9349              218         236                  0   
85.0          37.1659              222         236                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
82.0                  18              41             59        false    82  
83.0                   9              20             54        false    83  
84.0                  27              16             48        false    84  
85.0                  10              12             42        false    85  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
82.0          38.6379      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
86.0          29.6688              197         236                 20   
87.0          34.2473              209         236                  0   
88.0           36.637              215         236                  0   
89.0          38.7493              220         236                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
86.0                  28              31             57        false    86  
87.0                   5              18             61        false    87  
88.0                  19              14             51        false    88  
89.0                  21               9             46        false    89  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
86.0          38.5046      

this is reportrepeat
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
90.0          31.3919              197         236                 20   
91.0          34.3808              213         236                  0   
92.0          34.0244              206         237                  0   
93.0           34.182              206         238                  0   

     sumwaiting-patients sumovercapacity unusedcapacity future-modus ticks  
90.0                   8              45             63        false    90  
91.0                  17              20             54        false    91  
92.0                  21              23             59        false    92  
93.0                   7              20             60        false    93  
before storing it in report =, we have:
old report:
<class 'pandas.core.frame.DataFrame'>
     average-distance covered-shelters sumshelters predicted-shelters  \
90.0          40.5381      